In [1]:
import numpy as np
import cirq

In [2]:
class QuditHadamard(cirq.SingleQubitGate):
    
    def __init__(self, dim):
        self.dim = dim
        super(QuditHadamard, self)
        
    def _qid_shape_(self):
        return (self.dim,)

    def _unitary_(self):
        unitary = np.zeros((self.dim, self.dim), dtype = np.complex_)
        w = np.exp(2 * np.pi * 1j / self.dim)
        
        for i in range(self.dim):
            for j in range(self.dim):
                unitary[i, j] = w ** (i * j) / np.sqrt(self.dim)
        
        return unitary

    def _circuit_diagram_info_(self, args):
        return '[H+]'


In [3]:
class QuditInverseHadamard(cirq.SingleQubitGate):
    
    def __init__(self, dim):
        self.dim = dim
        super(QuditInverseHadamard, self)
        
    def _qid_shape_(self):
        return (self.dim,)

    def _unitary_(self):
        unitary = np.zeros((self.dim, self.dim), dtype = np.complex_)
        w = np.exp(2 * np.pi * 1j / self.dim)
        
        #For inverse hadamard, H^-1[i][j] = H[i][dim - j] for j > 0
        for i in range(self.dim):
            for j in range(self.dim):
                unitary[i, j] = w ** (i * (self.dim - j)) / np.sqrt(self.dim)
        
        return unitary

    def _circuit_diagram_info_(self, args):
        return '[H-]'


In [4]:
#Let the control qudit have dimension d1, and the target qudit have dimension d2.
#Let the control qudit be in basis state |x>
#Given (d1, d2) and (p1, p2):
#this gate rotates each basis state |y> for the target qudit by exp(2*pi*i*(d1^p1) * xy / (d2^p2))

class ControlledRotation(cirq.Gate):
    
    def __init__(self, dims, pows):
        self.dims = dims
        self.pows = pows
        super(ControlledRotation, self)
    
    def _num_qubits_self(self):
        return 2
    
    def _qid_shape_(self):
        return self.dims

    def _unitary_(self):
        d1, d2 = self.dims
        p1, p2 = self.pows
        matSize = d1 * d2
        unitary = np.zeros((matSize, matSize), dtype = np.complex_)
        w = np.exp(2 * np.pi * 1j * (d1 ** p1) / (d2 ** p2))
        
        for i in range(matSize):
            control = i // d2
            target = i % d2
            unitary[i, i] = w ** (control * target)
    
        return unitary

    def _circuit_diagram_info_(self, args):
        return 'O', 'U'

In [5]:

class InverseControlledRotation(cirq.Gate):
    def __init__(self, dims, pows):
        self.dims = dims
        self.pows = pows
        super(InverseControlledRotation, self)
    
    def _num_qubits_self(self):
        return 2
    
    def _qid_shape_(self):
        return self.dims

    def _unitary_(self):
        d1, d2 = self.dims
        p1, p2 = self.pows
        matSize = d1 * d2
        unitary = np.zeros((matSize, matSize), dtype = np.complex_)
        w = np.exp(-2 * np.pi * 1j * (d1 ** p1) / (d2 ** p2))
        
        for i in range(matSize):
            control = i // d2
            target = i % d2
            unitary[i, i] = w ** (control * target)
    
        return unitary

    def _circuit_diagram_info_(self, args):
        return 'O', 'U-'

In [28]:
#Swap two qudits of same dimension
#Let n be dimension of qudit. Map index an + b to bn + a using permutation matrix

class SwapGate(cirq.Gate):
    def __init__(self, dim):
        self.dim = dim
        super(SwapGate, self)
        
    def _num_qubits_self(self):
        return 2
    
    def _qid_shape_(self):
        return (self.dim, self.dim)
    
    def _unitary_(self):
        unitary = np.zeros((self.dim**2, self.dim**2))
        
        for i in range(self.dim):
            for j in range(self.dim):
                start = i * self.dim + j
                end = j * self.dim + i
                
                unitary[end, start] = 1
                
        return unitary
                
    def _circuit_diagram_info_(self, args):
        return 'X', 'X'

In [30]:
def InverseQFT(circuit, qudits, dim):
    num_qudits = len(qudits)
    
    for i in range(num_qudits // 2):
        circuit.append(SwapGate(dim).on(qudits[i], qudits[num_qudits - i - 1]))
        
    for i in range(num_qudits - 1, -1, -1):
        for j in range(num_qudits - 1, i, -1):
            circuit.append(InverseControlledRotation((dim, dim), (0, j - i + 1)).on(qudits[j], qudits[i]))
            
        circuit.append(QuditInverseHadamard(dim).on(qudits[i]))

In [21]:
#Get initial circuit into correct qubit state with X gates
def InitializeQubits(circuit, qubits, val):
    num_qb = len(qubits)
    
    for i in range(num_qb):
        if (val >> (num_qb - i - 1) & 1):
            circuit.append(cirq.X.on(qubits[i]))
        
        else:
            circuit.append(cirq.I.on(qubits[i]))

In [27]:
def Encode(circuit, qubits, qudits, dim):
    num_qb = len(qubits)
    num_qd = len(qudits)
    
    for qd in qudits:
        circuit.append(QuditHadamard(dim).on(qd))
        
    for i in range(num_qb):
        for j in range(num_qd):
            circuit.append(ControlledRotation((2, dim), (num_qb - i - 1, j + 1)).on(qubits[i], qudits[j]))

In [36]:
def getDirac(circuit):
    s = cirq.Simulator()
    results = s.simulate(circuit)
    
    return results.dirac_notation()

In [59]:
#Example encoding bit state 110 to qutrit state 20
#In the final circuit, the first 3 most significant digits are qubit states, while the last 2 digits are qutrit states

circuit = cirq.Circuit()
qubits = cirq.LineQid.for_qid_shape([2]*3, 0, 1)
qudits = cirq.LineQid.for_qid_shape([3]*2, 3, 1)

InitializeQubits(circuit, qubits, 6)
print("Initial state with qubits: {}".format(getDirac(circuit)))
print(circuit)

Encode(circuit, qubits, qudits, 3)
InverseQFT(circuit, qudits, 3)
print("\n\nState after encoding: {}".format(getDirac(circuit)))
print(circuit)

Initial state with qubits: |110⟩
0 (d=2): ───X───

1 (d=2): ───X───

2 (d=2): ───I───


State after encoding: |11020⟩
                       ┌──┐   ┌──┐
0 (d=2): ───X──────O────O───────────────────────────────────────
                   │    │
1 (d=2): ───X──────┼────┼O─────O────────────────────────────────
                   │    ││     │
2 (d=2): ───I──────┼────┼┼─────┼O────O──────────────────────────
                   │    ││     ││    │
3 (d=3): ───[H+]───U────┼U─────┼U────┼───X──────────U-───[H-]───
                        │      │     │   │          │
4 (d=3): ───[H+]────────U──────U─────U───X───[H-]───O───────────
                       └──┘   └──┘


In [50]:
#Verify every 3 qubit to 2 qutrit encoding
#In the final state, the first 3 most significant digits are qubit states, while the last 2 digits are qutrit states

num_qb = 3
num_qd = 2
dim = 3 

for val in range(2**num_qb):
    circuit = cirq.Circuit()
    qubits = cirq.LineQid.for_qid_shape([2]*num_qb, 0, 1)
    qudits = cirq.LineQid.for_qid_shape([dim]*num_qd, num_qb, 1)

    InitializeQubits(circuit, qubits, val)
    initial_state = getDirac(circuit)

    Encode(circuit, qubits, qudits, dim)
    InverseQFT(circuit, qudits, dim)
    final_state = getDirac(circuit)
    
    print("Initial state: {}        Final state: {}".format(initial_state, final_state))

Initial state: |000⟩        Final state: |00000⟩
Initial state: |001⟩        Final state: |00101⟩
Initial state: |010⟩        Final state: |01002⟩
Initial state: |011⟩        Final state: |01110⟩
Initial state: |100⟩        Final state: |10011⟩
Initial state: |101⟩        Final state: |10112⟩
Initial state: |110⟩        Final state: |11020⟩
Initial state: |111⟩        Final state: |11121⟩


In [56]:
#Verify every 2 qubit to 1 ququart encoding
num_qb = 2
num_qd = 1
dim = 4

for val in range(2**num_qb):
    circuit = cirq.Circuit()
    qubits = cirq.LineQid.for_qid_shape([2]*num_qb, 0, 1)
    qudits = cirq.LineQid.for_qid_shape([dim]*num_qd, num_qb, 1)

    InitializeQubits(circuit, qubits, val)
    initial_state = getDirac(circuit)

    Encode(circuit, qubits, qudits, dim)
    InverseQFT(circuit, qudits, dim)
    final_state = getDirac(circuit)
    
    print("Initial state: {}        Final state: {}".format(initial_state, final_state))

Initial state: |00⟩        Final state: |000⟩
Initial state: |01⟩        Final state: |011⟩
Initial state: |10⟩        Final state: |102⟩
Initial state: |11⟩        Final state: |113⟩


In [57]:
#Verify every 4 qubit to 3 qutrit encoding
num_qb = 4
num_qd = 3
dim = 3

for val in range(2**num_qb):
    circuit = cirq.Circuit()
    qubits = cirq.LineQid.for_qid_shape([2]*num_qb, 0, 1)
    qudits = cirq.LineQid.for_qid_shape([dim]*num_qd, num_qb, 1)

    InitializeQubits(circuit, qubits, val)
    initial_state = getDirac(circuit)

    Encode(circuit, qubits, qudits, dim)
    InverseQFT(circuit, qudits, dim)
    final_state = getDirac(circuit)
    
    print("Initial state: {}        Final state: {}".format(initial_state, final_state))

Initial state: |0000⟩        Final state: |0000000⟩
Initial state: |0001⟩        Final state: |0001001⟩
Initial state: |0010⟩        Final state: |0010002⟩
Initial state: |0011⟩        Final state: |0011010⟩
Initial state: |0100⟩        Final state: |0100011⟩
Initial state: |0101⟩        Final state: |0101012⟩
Initial state: |0110⟩        Final state: |0110020⟩
Initial state: |0111⟩        Final state: |0111021⟩
Initial state: |1000⟩        Final state: |1000022⟩
Initial state: |1001⟩        Final state: |1001100⟩
Initial state: |1010⟩        Final state: |1010101⟩
Initial state: |1011⟩        Final state: |1011102⟩
Initial state: |1100⟩        Final state: |1100110⟩
Initial state: |1101⟩        Final state: |1101111⟩
Initial state: |1110⟩        Final state: |1110112⟩
Initial state: |1111⟩        Final state: |1111120⟩
